In [ ]:
import pandas as pd

df = pd.read_csv('/content/Online Retail.csv')
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 8:26,3.39,17850.0,United Kingdom


In [ ]:
import pandas as pd

df = pd.read_csv('/content/Online Retail.csv')
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [ ]:
#상세 정보 보기
import pandas as pd

df = pd.read_csv('/content/Online Retail.csv')


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [ ]:
#컬럼별 누락값 개수 확인
import pandas as pd

df = pd.read_csv('/content/Online Retail.csv')

df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [ ]:
#고객ID 컬럼의 누락행 제거
#Quantity, UnitPrice 컬럼의 음수 행 제거
import pandas as pd
import numpy as np

df = pd.read_csv('/content/Online Retail.csv')

df = df[df.CustomerID.notnull()]
df = df[df.Quantity > 0]
df = df[df.UnitPrice > 0]
df.CustomerID = df.CustomerID.astype('category')
df['CheckoutPrice'] = df.UnitPrice * df.Quantity

def downcast_float(column):
  df[column] = pd.to_numeric(df[column], downcast='float')

def downcast_int(column):
  df[column] = pd.to_numeric(df[column], downcast='integer')

downcast_float('UnitPrice')
downcast_int('CustomerID')
downcast_int('Quantity')

df.to_csv('onlineRetial_cleaned.csv', index=False)

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
COLORS = sns.color_palette()

retail = pd.read_csv('/content/onlineRetial_cleaned.csv')

#InvoiceData 컬럼 날짜 데이터로 변경하기
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'], infer_datetime_format = True)
#retail.info()

#전체 매출
# total_revenue = retail.CheckoutPrice.sum()
# total_revenue

#국가별 매출
rev_by_countries = retail.groupby('Country').sum()['CheckoutPrice'].sort_values()
#rev_by_countries

#국가별 매출 시각화
# rev_by_countries.plot(figsize=(20, 10), kind='bar', color=COLORS[-1])

# plt.xlabel('Country', size='large')
# plt.ylabel('Revenue by countries', size='x-large')

# plt.xticks(rotation=45)

#국가별 매출 비율
# rev_by_countries / total_revenue

#막대그래프 그리기 함수 제작
def plot_bar(sr, xlabel, ylabel, title, figsize=(20, 10), color=COLORS[-1], rotation=45):

  sr.plot(figsize=figsize, kind='bar', color=color)

  plt.xlabel(xlabel, size='large')
  plt.ylabel(ylabel, size='large')

  plt.title(title, size='x-large')

  plt.xticks(rotation=rotation)

# plot_bar(rev_by_countries, 'Country','Revenue', 'Title')

#연월별 매출

#year, month 컬럼 추가
retail.insert(5, 'Year', retail.InvoiceDate.dt.year)
retail.insert(6, 'Month', retail.InvoiceDate.dt.month)
retail.head()

#year, month를 컬럼으로 그룹핑
rev_by_ym2 = retail.groupby(['Year','Month']).sum()[['CheckoutPrice']]
rev_by_ym2

#연월별 시각화
#plot_bar(rev_by_ym2 , 'Month', 'Revenue',' Revenue by month')

#요일별 매출
retail.insert(7, 'DayofWeek', retail.InvoiceDate.dt.dayofweek)
retail.head()

#그룹화
rev_by_dow2 = retail.groupby('DayofWeek').sum()[['CheckoutPrice']]
rev_by_dow2

#인덱스 바꾸기
rev_by_dow2.index = ['Mon','Tue','Wed', 'Thu', ' Fri', 'Sun']

#요일별 시각화
#plot_bar(rev_by_dow2 , 'Month', 'Revenue',' Revenue by month')

#hour추가
retail.insert(8, 'Hour', retail.InvoiceDate.dt.hour)
#retail

#hour칼럼으로 그룹핑
rev_by_hour2 = retail.groupby('Hour').sum()[['CheckoutPrice']]
rev_by_hour2

#시간별 시각화
#plot_bar(rev_by_hour2 , 'hour', 'Revenue',' Revenue by hour')

#TOP 10 판매 제품
top_selling = retail.groupby('StockCode').sum()['Quantity'].sort_values(ascending=False)[:10]
#top_selling

#판매 TOP 3 제품의 월별 판매량 추이

#상위 3개 인덱스 저장
top3 = top_selling.index.values[:3]

monthly_top3 = retail.groupby(['StockCode', 'Year','Month']).sum().loc[top3]
#monthly_top3

#상위 3개 인덱스 시각화
#plot_bar(monthly_top3.CheckoutPrice , 'Product/ym', 'Revenue',' Revenue of top3 items')


#고객별 첫 구매 연도 추출

#CustomerID로 그룹핑하여 InvoiceDate 컬럼의 최솟값을 구해 transform으로 기존 인덱스를 유지한 채 첫 구매 일자를 추출
retail.groupby('CustomerID')['InvoiceDate'].transform(np.min)
Start_Year = retail.groupby('CustomerID')['InvoiceDate'].transform(np.min).dt.year
#Start_Year

#고객별 첫 구매 월 추출
Start_Month = retail.groupby('CustomerID')['InvoiceDate'].transform(np.min).dt.month
Start_Month

#첫 구매 연, 월 컬럼으로 추가
retail.insert(7, 'Start_Year', Start_Year)
retail.insert(8, 'Start_Month', Start_Month)
#retail.tail()

#첫 구매후 몇 개월만의 구매인지 찾기
Month_Passed = (retail.Year - retail.Start_Year) * 12 + (retail.Month - retail.Start_Month)

#Month_Passed

#Month_Passed 컬럼 추가
retail.insert(9, 'Month_Passed', Month_Passed)

def make_month(month):
  if month < 10:
    return '0' + str(month)
  return str(month)
        
retail.Start_Month.apply(make_month)

Start_ym = retail.Start_Year.astype(str) + retail.Start_Month.apply(make_month)
retail.insert(9, 'Start_ym', Start_ym)

#Start_ym과 Month_Passed 컬럼으로 그룹핑
retail.groupby(['Start_ym','Month_Passed']).first()

#그룹별로 CustomerID 고유값 구하기
def get_unique_id(x):
  return np.unique(x)

retail.groupby(['Start_ym', 'Month_Passed'])['CustomerID'].apply(get_unique_id)

#그룹별 CustomerID의 고유값 개수 구하기
def get_unique_len_id(x):
  return len(np.unique(x))

retail.groupby(['Start_ym','Month_Passed'])['CustomerID'].apply(get_unique_len_id)


#데이터 프레임으로 전환
cohort_df = retail.groupby(['Start_ym', 'Month_Passed'])['CustomerID'].apply(get_unique_len_id).reset_index()

#Month_Passed를 컬럼으로 이동
cohort_df = cohort_df.pivot(index='Start_ym', columns='Month_Passed', values='CustomerID')

#Month_Passed가 0인 컬럼으로 정규화 후 반올림
cohort_df = round(cohort_df.div(cohort_df.iloc[:, 0],axis=0),4)*100

#heatmap
#sns.heatmap(cohort_df,linewidths=0.1, cmap='Blues', linecolor='white', annot=True)
#plt.show()

#우수 고객 찾기

#고객을 기준으로 해야하니 CustomerID로 그룹화한뒤 count메서드를 이용한다
#구매 횟수 기준 우수 고객
retail.groupby('CustomerID').count()
#내림차순
retail.groupby('CustomerID').count()['Quantity'].sort_values(ascending=False)

#구매 금액 기준 우수고객
retail.groupby('CustomerID').sum()['CheckoutPrice'].sort_values(ascending=False)


#몇시에 push 마케팅하는게 좋은지

#시간대 별 그룹핑
order_by_hour = retail.groupby('Hour').count()['Quantity']
order_by_hour

#시간대 별 주문 시각화
#plot_bar(order_by_hour , 'Hour', 'Orders',' Order by Hour')

#30분 함수
def get_half_an_hour(date):
  minute = ':00'

  if date.minute > 30:
    minute = ':30'

  hour = str(date.hour)

  if date.hour < 10:
    hour = '0' + hour

  return hour + minute

#30분 컬럼 추가
half_an_hour = retail.InvoiceDate.apply(get_half_an_hour)
retail.insert(6, 'Half_an_hour', half_an_hour)

#30분 컬럼으로 그룹핑
order_by_half_an_hour = retail.groupby('Half_an_hour').count()['Quantity']
order_by_half_an_hour

#30분 기준 시각화
#plot_bar(order_by_half_an_hour , 'Hour', 'Orders',' Order by half_an_hour')

(order_by_half_an_hour / order_by_half_an_hour.sum()) * 100

#개인 맞춤형 시간대별 주문량 파악
#CustomerID 와 Hour 컬럼으로 그룹핑
retail.groupby(['CustomerID', 'Hour']).count()

order_count_by_hour = retail.groupby(['CustomerID','Hour']).count()['Quantity']
order_count_by_hour

#데이터프레임화
order_count_by_hour = order_count_by_hour.reset_index().rename(columns={'Quantity':'Orders'})

#최대값 구하기 함수
get_max = lambda x: x.loc[x.Orders.idxmax()]

order_count_by_hour.groupby('CustomerID').apply(get_max)

result = order_count_by_hour.groupby('CustomerID').apply(get_max).reset_index(drop=True)

#Hour로 그룹핑
result.groupby('Hour').groups

#key
idx = result.groupby('Hour').groups[7].values
idx

result.loc[idx]

,CustomerID,Hour,Orders
73,12435,7,19
269,12678,7,61
319,12736,7,4
344,12775,7,15
375,12823,7,1
893,13538,7,5
1667,14619,7,20
2317,15505,7,29
